# Stanford CME 241 (Winter 2024) - Assignment 5

**Due: Feb 12 @ 11:59pm Pacific Time on Gradescope.**

Assignment instructions:
- **Solve any 3 of the 4 questions.**
- Empty code blocks are for your use. Feel free to create more under each section as needed.

Submission instructions:
- When complete, fill out your publicly available GitHub repo file URL and group members below, then export or print this .ipynb file to PDF and upload the PDF to Gradescope.

*Link to this ipynb file in your public GitHub repo (replace below URL with yours):* 

https://github.com/my-username/my-repo/assignment-file-name.ipynb

*Group members (replace below names with people in your group):* 

-Handi Zhao(hdzhao@stanford.edu);

-Sylvia Sun(ys3835@stanford.edu);

-Zhengji Yang(yangzj@stanford.edu)

## Imports

In [ ]:
!git clone https://github.com/TikhonJelvis/RL-book.git
import os
import sys
os.chdir('RL-book/rl')
sys.path.append('/content/RL-book')
import numpy as np
import itertools
from typing import Callable, Tuple, Iterable
from rl.distribution import Gaussian, Choose, SampledDistribution
from rl.markov_decision_process import MarkovDecisionProcess, State, TransitionStep
from rl.function_approx import LinearFunctionApprox, FunctionApprox
from rl.approximate_dynamic_programming import value_iteration
from rl.policy import Policy


## Question 1
You are a milkvendor and your task is to bring to your store a supply
(denoted $S \in \mathbb{R}$) of milk volume in the morning that will
give you the best profits. You know that the demand for milk through the
course of the day is a probability distribution function $f$ (for
mathematical convenience, assume people can buy milk in volumes that are
real numbers, hence milk demand $x \in \mathbb{R}$ is a continuous
variable with a probability density function). For every extra gallon of
milk you carry at the end of the day (supply $S$ exceeds random demand
$x$), you incur a cost of $h$ (effectively the wasteful purchases
amounting to the difference between your purchase price and end-of-day
discount disposal price since you are not allowed to sell the same milk
the next day). For every gallon of milk that a customer demands that you
don't carry (random demand $x$ exceeds supply $S$), you incur a cost of
$p$ (effectively the missed sales revenue amounting to the difference
between your sales price and purchase price). So your task is to
identify the optimal supply $S$ that minimizes your Expected Cost
$g(S)$, given by the following:

$$g_1(S) = E[\max(x-S, 0)] = \int_{-\infty}^{\infty} \max(x-S, 0) \cdot f(x) \cdot dx = \int_S^{\infty} (x-S) \cdot f(x) \cdot dx$$
$$g_2(S) = E[\max(S-x, 0)] = \int_{-\infty}^{\infty} \max(S-x, 0) \cdot f(x) \cdot dx = \int_{-\infty}^S (S-x) \cdot f(x) \cdot dx$$

$$g(S) = p \cdot g_1(S) + h \cdot g_2(S)$$

After you solve this problem, see if you can frame this problem in terms
of a call/put options portfolio problem.

#### Question 1 Answer:

By definition, optimal supply $S^*$ is defined by
\begin{align}
S^* &= \text{argmin}_{S}\,\, g(S)\\
    &= \text{argmin}_{S}\,\, p \cdot g_1(S) + h \cdot g_2(S)
\end{align}

As it's clear that both $g_1$ and $g_2$ are differentiable w.r.t. $S$, thus, $S^*$ satisfies the following first-order condition:
\begin{align}
\dfrac{\partial g(S^*)}{\partial S} &= \dfrac{\partial [p \cdot g_1(S^*) + h \cdot g_2(S^*)]}{\partial S} \\
                    &= p \cdot \int_{S^*}^\infty \dfrac{\partial (x - S^*)\cdot f(x)}{\partial S} dx + 
  h \cdot \int_{-\infty}^{S^*} \dfrac{\partial (S^* - x)\cdot f(x)}{\partial S} dx \\
&= - p \cdot \int_{S^*}^\infty f(x) dx + h \cdot \int_{-\infty}^{S^*} f(x) dx\\
&= 0
\end{align}

Let $F(x) = \int^x_{-\infty} f(s) ds$ be the cumulative distribution function of x, then the above formula can be rewritten as

\begin{equation}
- p (1 - F(S^*)) + h F(S^*) = 0,
\end{equation}

i.e.

\begin{equation}
(p + h) \cdot F(S^*) = p
\end{equation}


Thus, optimal supply $S^*$ is
$$ S^* = F^{-1}(\dfrac{p}{h + p}) $$

Call/Put Option Portfolio Problem:

For an underlying X with pdf $f$, the premiums for the European call and put option on $X$ are denoted by $p$ and $h$, i.e. $p+1$ and $h+1$ signify the ratios of the option price over its value. Now, if a customer aims to hedge against the volatility of X by constructing a portfolio comprising long 1 share of the call option and long 1 share of the put option, what is the optimal strike price $S$ such that the overall premium is minimized?

## Question 2
[rl/chapter8/optimal_bin_tree.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/chapter8/optimal_exercise_bin_tree.py)
models the American Payoff pricing problem as a `FiniteMarkovDecisionProcess` in the form of a binary tree with only two
discrete transitions for any given asset price. In the world of mathematical and computational finance, it is common practice to work
with continuous-valued asset prices and transitions to a continuous set of asset prices for the next time step. Your task is to model this
problem as a `MarkovDecisionProcess` (not finite) with discrete time, continuous-valued asset prices and a continuous-set of transitions.
Assume an arbitrary probability distribution for asset price movements from one time step to another, so you'd be sampling from the arbitrary
transition probability distribution. Hence, you will be solving this problem with Approximate Dynamic Programming using the code in
[rl/approximate_dynamic_programming.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/approximate_dynamic_programming.py).

In [ ]:
class ContinuousAssetPriceState(State):
    def __init__(self, price: float):
        self.price = price

class TradingAction(float):
    pass

class ContinuousAssetTradingMDP(MarkovDecisionProcess[ContinuousAssetPriceState, TradingAction]):
    def __init__(self, rate: float, vol: float, payoff: Callable[[float], float], action_range: Tuple[float, float]):
        self.rate = rate
        self.vol = vol
        self.payoff = payoff
        self.action_range = action_range

    def step(self, state: ContinuousAssetPriceState, action: TradingAction) -> SampledDistribution[Tuple[ContinuousAssetPriceState, float]]:
        # Transition model for MDP
        def next_state_reward_sampler():
            next_price_mean = state.price * np.exp(self.rate - 0.5 * self.vol ** 2)
            next_price_std = state.price * self.vol
            next_price = np.random.normal(next_price_mean, next_price_std)
            action_value = max(min(float(action), self.action_range[1]), self.action_range[0])
            reward = self.payoff(next_price) * action_value
            next_state = ContinuousAssetPriceState(price=next_price)
            return next_state, reward
        return SampledDistribution(next_state_reward_sampler)

    def actions(self, state: ContinuousAssetPriceState) -> Iterable[TradingAction]:
        num_actions = 100
        action_step = (self.action_range[1] - self.action_range[0]) / num_actions
        return (TradingAction(a) for a in np.arange(self.action_range[0], self.action_range[1], action_step))

def create_linear_function_approximator() -> FunctionApprox[ContinuousAssetPriceState]:
    # Linear function approximator
    return LinearFunctionApprox.create(feature_functions=[lambda s: s.price, lambda s: 1.0])

def non_terminal_states_distribution(mean: float, std_dev: float, num_samples: int) -> Choose[ContinuousAssetPriceState]:
    samples = np.random.normal(mean, std_dev, num_samples)
    return Choose([ContinuousAssetPriceState(price) for price in samples])


rate = 0.05
vol = 0.2
strike = 100
payoff = lambda price: max(price - strike, 0)
mdp = ContinuousAssetTradingMDP(rate, vol, payoff, action_range=(0, 1e4))

function_approximator = create_linear_function_approximator()

mean_price = 100
std_dev_price = 10
num_state_samples = 20

gamma = 0.95
num_iterations = 10

# Value iteration process
optimal_value_function_iterator = value_iteration(
    mdp=mdp,
    γ=gamma,
    approx_0=function_approximator,
    non_terminal_states_distribution=non_terminal_states_distribution(mean_price, std_dev_price, num_state_samples),
    num_state_samples=num_state_samples
)

# Optimal value function retrieval
optimal_value_function = next(itertools.islice(optimal_value_function_iterator, num_iterations, None))

# Result display
current_price = 100
state = ContinuousAssetPriceState(current_price)
estimated_value = optimal_value_function(state)
print(f"Estimated value at price {current_price}: {estimated_value}")


Estimated value at price 100: 0.20144928564100928


## Question 3
We'd like to build a simple simulator of Order Book Dynamics as a
`MarkovProcess` using the code in
[rl/chapter9/order_book.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/chapter9/order_book.py).
An object of type `OrderBook` constitutes the *State*. Your task is to
come up with a simple model for random arrivals of Market Orders and
Limit Orders based on the current contents of the `OrderBook`. This
model of random arrivals of Marker Orders and Limit Orders defines the
probabilistic transitions from the current state (`OrderBook` object) to
the next state (`OrderBook` object). Implement the probabilistic
transitions as a `MarkovProcess` and use it's `simulate` method to
complete your implementation of a simple simulator of Order Book
Dynamics.

Experiment with different models for random arrivals of Market Orders
and Limit Orders.

## Question 4
Derive the expressions for the Optimal Value Function and Optimal Policy
for the *Linear-Percentage Temporary* (LPT) Price Impact Model
formulated by Bertsimas and Lo. The LPT model is described below for all
$t = 0, 1, \ldots T-1$:

$$P_{t+1} = P_t \cdot e^{Z_t}$$ 

$$X_{t+1} = \rho \cdot X_t + \eta_t$$

$$Q_t = P_t \cdot (1 - \beta \cdot N_t - \theta \cdot X_t)$$ 

where $Z_t$ are independent and identically distributed random variables with mean
$\mu_Z$ and variance $\sigma^2_Z$ for all $t = 0, 1, \ldots, T-1$,
$\eta_t$ are independent and identically distributed random variables
with mean 0 for all $t = 0, 1, \ldots, T-1$, $Z_t$ and $\eta_t$ are
independent of each other for all $t = 0, 1, \ldots, T-1$, and
$\rho, \beta, \theta$ are given constants. The model assumes no
risk-aversion (Utility function is the identity function) and so, the
objective is to maximize the Expected Total Sales Proceeds over the
finite-horizon up to time $T$ (discount factor is 1). In your
derivation, use the same methodology as we followed for the *Simple
Linear Price Impact Model with no Risk-Aversion*.

Implement this LPT model by customizing the class
`OptimalOrderExecution` in [rl/chapter9/optimal_order_execution.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/chapter9/optimal_order_execution.py).

Compare the obtained Optimal Value Function and Optimal Policy against
the closed-form solution you derived above.

#### Question 4 Answer:
Using the definition in the linear impact with no risk-aversion model, we have states $s_t := (P_t, R_t, X_t) \in \mathcal{S}_t$ and actions $a_t := N_t \in \mathcal{A}_t$. Similarly, we also have $R_t=N-\sum_{i=0}^{t-1}N_i$, $R_0=N$. $\forall \, t<T$, $R_{t+1}=R_t-N_t$ and $N_{T-1}=R_{T-1}$. Sales proceeds in time step $t$ is $N_t \cdot Q_t=N_t\cdot P_t(1-\beta N_t-\theta X_t))$. We need to find the policy $\pi_t^*((P_t, R_t))=N_t^*$ that maximize $$\mathbb{E} \left[ \sum_{t=0}^{T-1} U(N_t, Q_t) \right]$$
    Value function for policy $\pi$ is $$V_t^{\pi}((P_t, R_t))=\mathbb{E}_{\pi} \left[ \sum_{i=t}^T N_i \cdot P_i(1-\beta N_i -\theta X_i)\right]$$ and the optimal value function is $\displaystyle V_t^*((P_t, R_t))= \max_{\pi} V_t^{\pi}((P_t, R_t))$. The optimal value function satisfy the Bellman Equation $$V_t^*((P_t, R_t))=\max_{N_t} \{N_t \cdot P_t(1-\beta N_t -\theta X_t)+\mathbb{E}[V_{t+1}^*((P_{t+1}, R_{t+1}))]\}$$
In this case, 
\begin{align*}
    V_{T-1}^*((P_{T-1}, R_{T-1})) &= N_{T-1} \cdot P_{T-1}(1-\beta N_{T-1} -\theta X_{T-1}) \\
    &=R_{T-1} \cdot P_{T-1}(1-\beta R_{T-1} -\theta X_{T-1})
\end{align*}
Then,
\begin{align*}
    V_{T-2}^*((P_{T-2}, R_{T-2})) =& \max_{N_{T-2}} \{N_{T-2} \cdot P_{T-2}(1-\beta N_{T-2} -\theta X_{T-2}) + \mathbb{E}[R_{T-1} \cdot P_{T-1}(1-\beta R_{T-1} -\theta X_{T-1})]\} \\
    =& \max_{N_{T-2}} \{N_{T-2} \cdot P_{T-2}(1-\beta N_{T-2} -\theta X_{T-2}) + \mathbb{E}[(R_{T-2}-N_{T-2}) \cdot P_{T-2}e^{Z_{T-2}}(1-\beta (R_{T-2}-N_{T-2}) -\theta (\rho X_{T-2}+\eta_{T-2})]\} \\
    =& \max_{N_{T-2}} \{N_{T-2} \cdot P_{T-2}(1-\beta N_{T-2} -\theta X_{T-2}) + (R_{T-2}-N_{T-2})P_{T-2}e^{\mu_Z+\sigma^2_Z/2} -\beta (R_{T-2}-N_{T-2})^2P_{T-2}e^{\mu_Z+\sigma^2_Z/2} \\
    &-\theta \rho X_{T-2}(R_{T-2}-N_{T-2})P_{T-2}e^{\mu_Z+\sigma^2_Z/2}\} \\
\end{align*}
Taking partial derivatives w.r.t. $N_{T-2}$, we have $$P_{T-2}(1-2\beta N_{T-2}-\theta X_{T-2}-e^{\mu_Z+\sigma^2_Z/2}+2\beta e^{\mu_Z+\sigma^2_Z/2} R_{T-2}-2\beta e^{\mu_Z+\sigma^2_Z/2}N_{T-2}+\theta \rho X_{T-2}e^{\mu_Z+\sigma^2_Z/2})=0$$ so that $$N_{T-2}=\frac{\theta(\rho e^{\mu_Z+\sigma^2_Z/2}-1)}{2\beta(e^{\mu_Z+\sigma^2_Z/2}+1)}X_{T-2}+\frac{e^{\mu_Z+\sigma^2_Z/2}}{e^{\mu_Z+\sigma^2_Z/2}+1}R_{T-2}+\frac{1-e^{\mu_Z+\sigma^2_Z/2}}{2\beta(1+e^{\mu_Z+\sigma^2_Z/2})}$$ and $$V_{T-2}^*((P_{T-2}, R_{T-2})) = e^{\mu_Z+\sigma^2_Z/2} P_{T-2} (c^{(4)}_{T-2} + c^{(5)}_{T-2} R_{T-2} + c^{(6)}_{T-2} X_{T-2} + c^{(7)}_{T-2} R_{T-2}^2 + c^{(8)}_{T-2} X_{T-2}^2 + c^{(9)}_{T-2} R_{T-2} X_{T-2})$$
Continuing backwards in time in this manner gives: 
$$N_t^* = c^{(1)}_t + c^{(2)}_t R_t + c^{(3)}_t X_t $$

\begin{align*}
    V^*_t((P_t,R_t,X_t)) = e^{\mu_Z + \frac {\sigma_Z^2} 2} \cdot P_t \cdot ( & c^{(4)}_t + c^{(5)}_t R_t + c^{(6)}_t X_t + c^{(7)}_t R_t^2 + c^{(8)}_t X_t^2 + c^{(9)}_t R_t X_t)\\
\end{align*}

In [1]:
from dataclasses import dataclass
from typing import Callable, Sequence, Tuple, Iterator
from rl.distribution import Distribution, SampledDistribution, Choose
from rl.function_approx import FunctionApprox, LinearFunctionApprox
from rl.markov_decision_process import MarkovDecisionProcess, NonTerminal, State
from rl.policy import DeterministicPolicy
from rl.approximate_dynamic_programming import back_opt_vf_and_policy, ValueFunctionApprox

import numpy as np

In [2]:
@dataclass(frozen=True)
class PriceSharesAndMarket:
    price: float
    market_factor: float
    shares: int


@dataclass(frozen=True)
class OptimalOrderExecution:
    shares: int
    time_steps: int
    avg_exec_price_diff: Sequence[Callable[[PriceSharesAndMarket], float]]
    price_dynamics: Sequence[Callable[[PriceSharesAndMarket], Distribution[float]]]
    market_dynamics: Sequence[Callable[[PriceSharesAndMarket], Distribution[float]]]
    utility_func: Callable[[float], float]
    discount_factor: float
    func_approx: ValueFunctionApprox[PriceSharesAndMarket]
    initial_price_distribution: Distribution[float]
    init_market_factor: float

    def get_mdp(self, t: int) -> MarkovDecisionProcess[PriceSharesAndMarket, int]:
        """
        State is (Price P_t, Remaining Shares R_t)
        Action is shares sold N_t
        """

        utility_f: Callable[[float], float] = self.utility_func
        price_diff: Sequence[Callable[[PriceSharesAndMarket], float]] = self.avg_exec_price_diff
        price_dynamics: Sequence[Callable[[PriceSharesAndMarket], Distribution[float]]] = self.price_dynamics
        market_dynamics: Sequence[Callable[[PriceSharesAndMarket], Distribution[float]]] = self.market_dynamics
        steps: int = self.time_steps

        class OptimalExecutionMDP(MarkovDecisionProcess[PriceSharesAndMarket, int]):

            def step(
                self,
                p_r: NonTerminal[PriceSharesAndMarket],
                sell: int
            ) -> SampledDistribution[Tuple[State[PriceSharesAndMarket],
                                           float]]:

                def sr_sampler_func(
                    p_r=p_r,
                    sell=sell
                ) -> Tuple[State[PriceSharesAndMarket], float]:
                    p_s: PriceSharesAndMarket = PriceSharesAndMarket(
                        price=p_r.state.price,
                        market_factor=p_r.state.market_factor,
                        shares=sell
                    )
                    next_price: float = price_dynamics[t](p_s).sample()
                    next_market: float = market_dynamics[t](p_s).sample()
                    next_rem: int = p_r.state.shares - sell
                    next_state: PriceSharesAndMarket = PriceSharesAndMarket(
                        price=next_price,
                        market_factor=next_market,
                        shares=next_rem
                    )
                    reward: float = utility_f(
                        sell * p_r.state.price * (1 - price_diff[t](p_s))
                    )
                    return (NonTerminal(next_state), reward)

                return SampledDistribution(
                    sampler=sr_sampler_func,
                    expectation_samples=100
                )

            def actions(self, p_s: NonTerminal[PriceSharesAndMarket]) -> \
                    Iterator[int]:
                if t == steps - 1:
                    return iter([p_s.state.shares])
                else:
                    return iter(range(p_s.state.shares + 1))

        return OptimalExecutionMDP()

    def get_states_distribution(self, t: int) -> \
            SampledDistribution[NonTerminal[PriceSharesAndMarket]]:

        def states_sampler_func() -> NonTerminal[PriceSharesAndMarket]:
            price: float = self.initial_price_distribution.sample()
            market_factor = self.init_market_factor
            rem: int = self.shares
            for i in range(t):
                sell: int = Choose(range(rem + 1)).sample()
                price = self.price_dynamics[i](PriceSharesAndMarket(
                    price=price,
                    market_factor=market_factor,
                    shares=rem
                )).sample()
                market_factor = self.market_dynamics[i](PriceSharesAndMarket(
                    price=price,
                    market_factor=market_factor,
                    shares=rem
                )).sample()
                rem -= sell
            return NonTerminal(PriceSharesAndMarket(
                price=price,
                market_factor=market_factor,
                shares=rem
            ))

        return SampledDistribution(states_sampler_func)

    def backward_induction_vf_and_pi(
        self
    ) -> Iterator[Tuple[ValueFunctionApprox[PriceSharesAndMarket],
                        DeterministicPolicy[PriceSharesAndMarket, int]]]:

        mdp_f0_mu_triples: Sequence[Tuple[
            MarkovDecisionProcess[PriceSharesAndMarket, int],
            ValueFunctionApprox[PriceSharesAndMarket],
            SampledDistribution[NonTerminal[PriceSharesAndMarket]]
        ]] = [(
            self.get_mdp(i),
            self.func_approx,
            self.get_states_distribution(i)
        ) for i in range(self.time_steps)]

        num_state_samples: int = 10000
        error_tolerance: float = 1e-6

        return back_opt_vf_and_policy(
            mdp_f0_mu_triples=mdp_f0_mu_triples,
            γ=self.discount_factor,
            num_state_samples=num_state_samples,
            error_tolerance=error_tolerance
        )


if __name__ == '__main__':

    from rl.distribution import Gaussian

    init_price_mean: float = 100.0
    init_price_stdev: float = 10.0
    init_market_factor = 0
    num_shares: int = 100
    num_time_steps: int = 5
    beta: float = 5e-7
    theta: float = 0.005
    rho = 0.5
    μ_z: float = 0
    σ_z: float = 0.02/np.sqrt(13)
    σ_eta: float = np.sqrt(1 - rho**2)

    price_diff = [lambda p_s: beta * p_s.shares + theta * p_s.market_factor for _ in range(num_time_steps)]
    market_dynamics = [lambda p_s: Gaussian(μ=rho * p_s.market_factor, σ=σ_eta) for _ in range(num_time_steps)]
    price_dynamics = [lambda p_s: Gaussian(μ=μ_z, σ=σ_z).map(lambda a: np.exp(a) * p_s.price) for _ in range(num_time_steps)]
    ffs = [
        lambda p_s: p_s.state.price * p_s.state.shares,
        lambda p_s: float(p_s.state.shares * p_s.state.shares)
    ]
    fa: FunctionApprox = LinearFunctionApprox.create(feature_functions=ffs)
    init_price_distrib: Gaussian = Gaussian(
        μ=init_price_mean,
        σ=init_price_stdev
    )

    ooe: OptimalOrderExecution = OptimalOrderExecution(
        shares=num_shares,
        time_steps=num_time_steps,
        avg_exec_price_diff=price_diff,
        price_dynamics=price_dynamics,
        market_dynamics=market_dynamics,
        utility_func=lambda x: x,
        discount_factor=1,
        func_approx=fa,
        initial_price_distribution=init_price_distrib,
        init_market_factor=init_market_factor,
    )
    it_vf: Iterator[Tuple[ValueFunctionApprox[PriceSharesAndMarket],
                          DeterministicPolicy[PriceSharesAndMarket, int]]] = ooe.backward_induction_vf_and_pi()

    state: PriceSharesAndMarket = PriceSharesAndMarket(
        price=init_price_mean,
        market_factor=init_market_factor,
        shares=num_shares
    )
    print("Backward Induction: VF And Policy")
    print("---------------------------------")
    print()
    for t, (vf, pol) in enumerate(it_vf):
        print(f"Time {t:d}")
        print()
        opt_sale: int = pol.action_for(state)
        val: float = vf(NonTerminal(state))
        print(f"Optimal Sales = {opt_sale:d}, Opt Val = {val:.3f}")
        print()
        print("Optimal Weights below:")
        print(vf.weights.weights)
        print()


Backward Induction: VF And Policy
---------------------------------

Time 0

Optimal Sales = 2, Opt Val = 10059.739

Optimal Weights below:
[1.00485747 0.00111643]

Time 1

Optimal Sales = 7, Opt Val = 10053.597

Optimal Weights below:
[1.00413703 0.00122269]

Time 2

Optimal Sales = 0, Opt Val = 10044.374

Optimal Weights below:
[1.00332429 0.00111313]

Time 3

Optimal Sales = 0, Opt Val = 10031.879

Optimal Weights below:
[1.00197266 0.00121522]

Time 4

Optimal Sales = 1, Opt Val = 10012.952

Optimal Weights below:
[0.99960358 0.00169166]

